# CSV dialect detection with CleverCSV

**Author**: G.J.J. van den Burg <gvandenburg@turing.ac.uk>

In this note we'll show some examples of using CleverCSV, a package for handling 
messy CSV files. 

We'll compare CleverCSV to the built-in Python CSV module and show how this is not as robust as CleverCSV on some examples. The examples will mainly show files where the built-in Python CSV module fails to detect the dialect correctly. These files are of course selected for this tutorial, because it wouldn't be very interesting to show files where both methods are correct. For more details on the science behind CleverCSV and a complete and fair comparison to other CSV packages, see the [paper](https://arxiv.org/abs/1811.11242).

The example CSV files all come from MIT-licensed GitHub repositories.

## Installation
First, download and install the development version of CleverCSV:
```
$ virtualenv ./clevercsv
$ cd ./clevercsv && . bin/activate
$ pip install poetry requests
$ git clone https://github.com/alan-turing-institute/CleverCSV
$ cd CleverCSV/python
$ poetry install
$ cd ../../../
```

If you want to follow along, I'll assume that you run this notebook in a location where the ``ccsv`` and ``requests`` modules are on the Python path.

## Setting up

In [2]:
import csv
import ccsv
import io
import requests

def page(url):
    """ Get the content of a webpage using requests, assuming UTF-8 encoding """
    page = requests.get(url)
    content = page.content.decode('utf-8')
    return content

def head(content, num=10):
    """ Preview a CSV file """
    print('--- File Preview ---')
    for i, line in enumerate(io.StringIO(content, newline=None)):
        print(line, end='')
        if i == num - 1:
            break
    print('\n---')
    
def sniff_url(content):
    """ Utility to run the python Sniffer on a CSV file at a URL """
    try:
        dialect = csv.Sniffer().sniff(content)
        print("CSV Sniffer detected: delimiter = %r, quotechar = %r" % (dialect.delimiter,
                                                                        dialect.quotechar))
    except csv.Error:
        print("No result from the Python CSV Sniffer")

def detect_url(content, verbose=True):
    """ Utility to run the CleverCSV detector on a CSV file at a URL """
    # We have designed CleverCSV to be a drop-in replacement for the CSV module
    try:
        dialect = ccsv.Sniffer().sniff(content, verbose=verbose)
        print("CleverCSV detected: delimiter = %r, quotechar = %r" % (dialect.delimiter, 
                                                                      dialect.quotechar))
    except ccsv.Error:
        print("No result from CleverCSV")
        
def test(url, verbose=False, n_preview=10):
    content = page(url)
    head(content, num=n_preview)
    print("\nRunning Python Sniffer")
    sniff_url(content)
    print("\nRunning CleverCSV:")
    detect_url(content, verbose=verbose)

## Example 1: No output from Python Sniffer

The first file we'll look at is a simple CSV file that uses the semicolon as delimiter. 

In [3]:
test("https://raw.githubusercontent.com/grezesf/Research/17b1e829d1d4b8954661270bd8b099e74bb45ce7/Reservoirs/Task0_Replication/code/preprocessing/factors.csv")

--- File Preview ---
log(E);de_log(E);dede_log(E);cepstrum;de_cepstrum;dede_cepstrum
0.27;1.77;4.97;0.1;0.61;1.75;
1.75;1.25;1.00;1.25;0.50;0.25;
---

Running Python Sniffer
No result from the Python CSV Sniffer

Running CleverCSV:
CleverCSV detected: delimiter = ';', quotechar = ''


As we can see, the Python CSV sniffer fails on this one, even though the formatting doesn't seem to be that uncommon. CleverCSV handles this file correctly.

## Example 2: Incorrect output from Python Sniffer

The next example is quite a long file with a lot of potential delimiters. In total, CleverCSV considers 180 different dialects on this file and determines the best dialect by computing a pattern score and a type score. The pattern score is related to how many cells we have per row given a certain dialect, and the type score reflects whether the cells in the parsed file have known data types (such as integer, date, string, etc.). 

If you want to see the output of CleverCSV while it runs the detection, you can set ``verbose=True`` in the next line.

In [4]:
test("https://raw.githubusercontent.com/agh-glk/pyconpl2013-nlp/37f6f50a45fc31c1a5ad25010fff681a8ce645b8/gsm.csv", verbose=False)

--- File Preview ---
name,price,state
NOWY DOTYKOWY LG T375 Wi-Fi DUAL SIM + GRATISY FV,238.0,new
NOWY SAMSUNG GALAXY NOTE 2 N7100 WHITE FV 23%,1799.0,new
NOKIA 3510 12miesięcy  GWARANCJI   Dostawa 24h,79.0,new
NOKIA 206 DUAL SIM GWAR. 2 LATA FV23% OKAZJA!,199.0,new
SAMSUNG B2100i SOLID BEZ SIML. +ETUI 24GW PL FV23%,298.0,new
Polski SAMSUNG S5300 GALAXY POCKET Gwar 24M-ce,318.0,new
Polski SAMSUNG S5610 Gwar 24M-ce KRAKÓW 5Mpx Foto,298.0,new
BIAŁY SAMSUNG I8190 GALAXY S III Mini Gwar 24M-ce,898.0,new
= SONY XPERIA E C1505 White/Biały!!! Wrocław =,475.0,new

---

Running Python Sniffer
CSV Sniffer detected: delimiter = '!', quotechar = '"'

Running CleverCSV:
CleverCSV detected: delimiter = ',', quotechar = '"'


Note that CleverCSV is a bit slower than the Python Sniffer. This is the focus of our ongoing development efforts and is also affected by running Python through Jupyter. But let's not forget that at least CleverCSV is correct!

## Conclusion

Below are some more examples, but I think you'll get the idea by now. CleverCSV is much more robust against messy CSV files and is an easy to use drop-in replacement for the Python csv module. Just replace ``import csv`` by ``import ccsv`` in your code!

We're still working on adding some more features to CleverCSV and speeding up and improving the dialect detection algorithm. One of the novel features that we added is a ``clevercsv`` executable with the following functions:

- ``detect`` to run dialect detection directly from the command line
- ``view`` to open a CSV file in a table viewer after automatic detection of the dialect
- ``standardize`` to convert a CSV file in a messy format to the standard CSV format

We hope you find CleverCSV useful! If you encounter any issues or files where CleverCSV fails, please leave a comment on GitHub.

## Further Examples

In [5]:
# No result from Python (note that this file says it uses "|" as separator, but actually uses ","!)
test("https://raw.githubusercontent.com/queq/just-stuff/c1b8714664cc674e1fc685bd957eac548d636a43/pov/TopFixed/build/project_r_pad.csv", n_preview=20)

--- File Preview ---
#Release 14.4 - par P.49d (lin)
#Copyright (c) 1995-2012 Xilinx, Inc.  All rights reserved.

#Thu Jun 20 07:23:42 2013

#
## NOTE: This file is designed to be imported into a spreadsheet program
# such as Microsoft Excel for viewing, printing and sorting. The |
# character is used as the data field separator. This file is also designed
# to support parsing.
#
#INPUT FILE:       project.ncd
#OUTPUT FILE:      project_r_pad.csv
#PART TYPE:        xc3s500e
#SPEED GRADE:      -5
#PACKAGE:          fg320
#
# Pinout by Pin Number:
# 
# -----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,-----,

---

Running Python Sniffer
No result from the Python CSV Sniffer

Running CleverCSV:
CleverCSV detected: delimiter = ',', quotechar = ''


In [6]:
# Python says '\r' (carriage return) is the delimiter!
test("https://raw.githubusercontent.com/HAYASAKA-Ryosuke/TodenGraphDay/8f052219d037edabebd488e5f6dc2ddbe8367dc1/juyo-j.csv")

--- File Preview ---
2013/12/12 23:20 UPDATE
ピーク時供給力(万kW),時間帯,供給力情報更新日,供給力情報更新時刻
4849,17:00〜18:00,12/12,8:30

予想最大電力(万kW),時間帯,予想最大電力情報更新日,予想最大電力情報更新時刻
4210,17:00〜18:00,12/12,8:30

DATE,TIME,当日実績(万kW),予測値（万kW)
2013/12/12,0:00,3098,0
2013/12/12,1:00,2948,0

---

Running Python Sniffer
CSV Sniffer detected: delimiter = '\r', quotechar = '"'

Running CleverCSV:
CleverCSV detected: delimiter = ',', quotechar = ''


In [7]:
# No result from Python csv
test("https://raw.githubusercontent.com/philipmcg/minecraft-service-windows/774892ff0c27a76b6db20ba3750149c19b7a3351/MinecraftService/MinecraftService/gcsv_sample.csv")

--- File Preview ---
// comment
 // another comment

~test_name,row_name,col1,col2
row1,r1c1,r1c2
row2,r2c1,r2c2
---

Running Python Sniffer
No result from the Python CSV Sniffer

Running CleverCSV:
CleverCSV detected: delimiter = ',', quotechar = ''


In [8]:
# Incorrect delimiter from Python csv
test("https://raw.githubusercontent.com/OptimusGitEtna/RestSymf/635e4ad8a288cde64b306126c986213de71a4f4a/Python-3.4.2/Doc/tools/sphinxext/susp-ignored.csv")

--- File Preview ---
c-api/arg,,:ref,"PyArg_ParseTuple(args, ""O|O:ref"", &object, &callback)"
c-api/list,,:high,list[low:high]
c-api/sequence,,:i2,del o[i1:i2]
c-api/sequence,,:i2,o[i1:i2]
c-api/unicode,,:end,str[start:end]
c-api/unicode,,:start,unicode[start:start+length]
distutils/examples,267,`,This is the description of the ``foobar`` package.
distutils/setupscript,,::,
extending/embedding,,:numargs,"if(!PyArg_ParseTuple(args, "":numargs""))"
extending/extending,,:myfunction,"PyArg_ParseTuple(args, ""D:myfunction"", &c);"

---

Running Python Sniffer
CSV Sniffer detected: delimiter = ' ', quotechar = '"'

Running CleverCSV:
CleverCSV detected: delimiter = ',', quotechar = '"'
